In [1]:
import torch
import numpy as np


# テストデータを生成する関数
def generate_test_data(batch_size=2, seq_len=5, embed_size=128):
    """テスト用のランダムデータを生成"""
    data = {
        "d": torch.randint(0, 30, (batch_size, seq_len)),
        "t": torch.randint(0, 24, (batch_size, seq_len)),
        "input_x": torch.randint(0, 200, (batch_size, seq_len)),
        "input_y": torch.randint(0, 200, (batch_size, seq_len)),
        "time_delta": torch.randint(0, 10, (batch_size, seq_len)),
        "label_x": torch.randint(0, 200, (batch_size, seq_len)),
        "label_y": torch.randint(0, 200, (batch_size, seq_len)),
        "len": torch.randint(1, seq_len + 1, (batch_size,)),
    }
    return data


# 95%信頼区間を計算する関数
def calculate_95_percentile(data, mask):
    """データの95%信頼区間を計算"""
    data_np = data[mask].cpu().numpy()
    lower_bound = np.percentile(data_np, 2.5)
    upper_bound = np.percentile(data_np, 97.5)
    return lower_bound, upper_bound

In [8]:
def test_prediction_on_test_data():
    # テストデータを生成
    data = generate_test_data()
    print("Input X:", data["input_x"])
    print("Input Y:", data["input_y"])

    # マスクを作成（簡単な例としてinput_xが100未満の場所をマスクとする）
    pred_mask = data["input_x"] < 100

    # input_xとinput_yの95%信頼区間を計算
    non_mask = ~pred_mask
    lower_x, upper_x = calculate_95_percentile(data["input_x"], non_mask)
    lower_y, upper_y = calculate_95_percentile(data["input_y"], non_mask)

    print(f"95% Confidence Interval for X: [{lower_x}, {upper_x}]")
    print(f"95% Confidence Interval for Y: [{lower_y}, {upper_y}]")

    # outputとしてランダムなテンソルを生成（実験のため）
    output = torch.rand(
        (data["input_x"].shape[0], data["input_x"].shape[1], 2, 200)
    )  # (Batch Size, Sequence Length, 2, 200)

    # 信頼区間に基づく予測値の修正処理
    for step in range(data["input_x"].shape[1]):

        # x座標が95%信頼区間外ならその要素を0にする
        output[step][0] = torch.where(
            (output[step][0] < lower_x) | (output[step][0] > upper_x),
            torch.zeros_like(output[step][0]),
            output[step][0],
        )

        # y座標が95%信頼区間外ならその要素を0にする
        output[step][1] = torch.where(
            (output[step][1] < lower_y) | (output[step][1] > upper_y),
            torch.zeros_like(output[step][1]),
            output[step][1],
        )

    print("Modified Output X (Step 0):", output[:, 0, 0])
    print("Modified Output Y (Step 0):", output[:, 0, 1])

    # 最大値のインデックスを確認
    pred_x = torch.argmax(output[:, 0, 0], dim=-1)
    pred_y = torch.argmax(output[:, 0, 1], dim=-1)

    print("Predicted X:", pred_x)
    print("Predicted Y:", pred_y)

In [17]:
output = torch.rand((30, 15, 2, 200))

In [18]:
output.shape

torch.Size([30, 15, 2, 200])

In [22]:
output[0][0][0].shape

torch.Size([200])

In [30]:
lower_x = 50
upper_x = 150

output[0][0][0][:lower_x] = 0
output[0][0][0][upper_x:] = 0

In [31]:
output[0][0][0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7006, 0.4941, 0.0978, 0.9069,
        0.1692, 0.7643, 0.9438, 0.0473, 0.8720, 0.9358, 0.7005, 0.1497, 0.2745,
        0.6766, 0.2225, 0.9912, 0.5649, 0.4029, 0.3275, 0.9184, 0.8596, 0.7537,
        0.8281, 0.1770, 0.0449, 0.0277, 0.8523, 0.2867, 0.8168, 0.1911, 0.2679,
        0.7751, 0.6985, 0.5829, 0.2618, 0.2281, 0.3669, 0.2978, 0.2145, 0.0736,
        0.5518, 0.6523, 0.0566, 0.4767, 0.4921, 0.3649, 0.4238, 0.6380, 0.7666,
        0.7555, 0.2252, 0.2933, 0.6332, 0.2961, 0.5434, 0.5188, 0.5876, 0.2590,
        0.8074, 0.8607, 0.4031, 0.8344, 

In [ ]:
torch.where(
    (output[0][0][0] < lower_x) | (output[0][0][0] > upper_x),
    torch.zeros_like(output[step][0]),  # 条件に一致する要素はゼロ
    output[step][0],  # 条件に一致しない要素はそのまま残す
)